In [144]:
from nlp_dataset import NLPDataset, pad_collate_fn
from utile_functions import load_C2V, load_glove, create_embedding

In [145]:
train_dataset = NLPDataset('data/text_twitter_raw.csv', ratio=0.7, exclude=None, senter=True, removeStop=False)
valid_dataset = NLPDataset('data/text_twitter_raw.csv', ratio=0.7, textVocab=train_dataset.textVocab,
                       labelVocab=train_dataset.labelVocab)

In [146]:
gloVe = load_glove()
c2v = load_C2V()
em = create_embedding(train_dataset.textVocab.stoi, gloVe, c2v)

In [147]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from abc import ABC
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [183]:
class LSTMModel(nn.Module, ABC):

    def __init__(self, embedding, lstm=[600, 300], fc=[600, 300, 150, 1], num_layer=6, bidirectional=True, dropout=0.5,
                 criterion=nn.BCEWithLogitsLoss()):
        super().__init__()
        dev = "cuda:0"
        device = torch.device(dev)

        self.embedding_matrix = embedding
        self.criterion = criterion
        l = list()
        for i in range(0, len(lstm) - 1):
            l.append(nn.LSTM(lstm[i], lstm[i + 1], num_layers=num_layer, bidirectional=bidirectional, dropout=dropout).to(
                device))
        self.lstms = nn.ModuleList(l)
        f = list()
        for i in range(0, len(fc) - 1):
            f.append(nn.Linear(in_features=fc[i], out_features=fc[i + 1], bias=True).to(device))
        self.fcs = nn.ModuleList(f)
        self.dropout = nn.Dropout(dropout)
        self.bestAcc=0.0
        self.bestF1=0.0

    def forward(self, X):
        dev = "cuda:0"
        device = torch.device(dev)
        texts, labels, lengths = X
        matrix_x = self.embedding_matrix(texts)
        matrix_x = matrix_x.transpose(1, 0)

        hidden = None
        out = matrix_x
        for lstm in self.lstms:
            out, hidden = lstm(out, hidden)
        h = out[-1]
        
        for fc in self.fcs:
            h = fc(h)
            if self.fcs[-1] != fc:
                h = torch.relu(h)
        return h

    def get_loss(self, x, y):

        return self.criterion(x, y)

    def train_model(self, train_data_loader, valid_data_loader, num_iter=20,  lr=0.0001, early_stop=False):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        best_val_loss=1000
        epoch_from_best=0
        best_model=None
        for epoch in range(0, num_iter):
            self.train()
            for i, data in enumerate(train_data_loader):
                texts, labels, lengths = data
                loss = self.get_loss(self.forward(data).squeeze(-1), labels.float())
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), 0.25)
                optimizer.step()
                optimizer.zero_grad()
            val_loss = self.evaluate(valid_data_loader, epoch)
            if val_loss<best_val_loss:
                best_val_loss=val_loss
                epoch_from_best=0
                best_model=self
            else:
                epoch_from_best+=1
            if epoch_from_best>=10 and early_stop:
                print('Finished because of early stop')
                break
        print(f'Best accuracy = {self.bestAcc}')
        print(f'Best f1 = {self.bestF1}')
        return best_model

    def evaluate(self, data_loader, epoch):
        self.eval()
        acc = 0
        f1 = 0
        recall = 0
        prec = 0
        counter = 0
        loss = 0
        with torch.no_grad():
            for i, data in enumerate(data_loader):
                texts, labels, lengths = data
                logits = self.forward(data).squeeze(-1)
                loss += self.get_loss(logits, labels.float())
                y_pred = torch.round(torch.sigmoid(logits))
                labels_np = labels.cpu().detach().numpy()
                y_np = y_pred.cpu().detach().numpy()
                acc += accuracy_score(labels_np, y_np)
                f1 += f1_score(labels_np, y_np)
                recall += recall_score(labels_np, y_np)
                prec += precision_score(labels_np, y_np)
                counter += 1
            ma=acc / counter
            mf = f1 / counter
            if ma>self.bestAcc:
                self.bestAcc=ma
            if mf>self.bestF1:
                self.bestF1=mf
            print(f'Epoch: {epoch}, eval loss= {loss / counter}')
            print(f'Accuracy= {ma}')
            print(f'f1= {mf}')
            print(f'Precision= {prec / counter}')
            print(f'Recall= {recall / counter}')
            print("-----------------------------")
            return loss/counter
    
    def test_model(self, data_loader):
        acc = 0
        f1 = 0
        recall = 0
        prec = 0
        counter = 0
        loss = 0
        for i, data in enumerate(data_loader):
            texts, labels, lengths = data
            logits = self.forward(data).squeeze(-1)
            loss += self.get_loss(logits, labels.float())
            y_pred = torch.round(torch.sigmoid(logits))
            labels_np = labels.cpu().detach().numpy()
            y_np = y_pred.cpu().detach().numpy()
            acc += accuracy_score(labels_np, y_np)
            f1 += f1_score(labels_np, y_np)
            recall += recall_score(labels_np, y_np)
            prec += precision_score(labels_np, y_np)
            counter += 1
        ma=acc / counter
        mf = f1 / counter
        if ma>self.bestAcc:
            self.bestAcc=ma
        if mf>self.bestF1:
            self.bestF1=mf
        
        print(f'Epoch: {epoch}, eval loss= {loss / counter}')
        print(f'Accuracy= {ma}')
        print(f'f1= {mf}')
        print(f'Precision= {prec / counter}')
        print(f'Recall= {recall / counter}')
        print("-----------------------------")

In [150]:
def load_glove_twitter():
    dev = "cuda:0"
    device = torch.device(dev)
    gloVe = {}
    vectors = open('data/glove.twitter.27B.200d.txt', "r", encoding="utf-8")
    for v in vectors:
        vector = v.split(" ")
        text = vector.pop(0)
        gloVe[text] = torch.tensor([float(num) for num in vector]).to(device)
    vectors.close()
    return gloVe

In [151]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                                  shuffle=True, collate_fn=pad_collate_fn)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=64,
                                  shuffle=True, collate_fn=pad_collate_fn)

In [166]:
from torch.nn import Embedding
def create_embedding_t(vocab, gloVe=None, c2v=None, twitter=False, c2v_size=300):
    if twitter:
        glove_size=200
    else:
        glove_size=300
    dev = "cuda:0"
    device = torch.device(dev)
    tensor_size = glove_size+c2v_size
    if gloVe is None:
        tensor_size -= glove_size
    if c2v is None:
        tensor_size -= c2v_size
    if gloVe is None and c2v is None:
        tensor_size = 300
        print("TU sam")
        return Embedding.from_pretrained(torch.randn((len(vocab), tensor_size)).to(device), padding_idx=0, freeze=False)

    embedding_matrix = torch.randn((len(vocab), tensor_size)).to(device)

    for index, word in enumerate(vocab):
        ten = None
        if gloVe is not None:
            glVec = torch.randn(glove_size).to(device)
        if c2v is not None:
            c2Vec = torch.randn(c2v_size).to(device)
        if word == '<PAD>':
            ten = torch.zeros((1, tensor_size)).to(device)
        else:
            glWord=word
            if twitter:
                if glWord == '@username':
                    glWord='<user>'
                if glWord.startswith('#'):
                    glWord = '<hashtag>'
                if glWord == 'HTTPLINK':
                    glWord = '<url>'
            if gloVe is not None and word in gloVe:
                glVec = gloVe[glWord]
            if c2v is not None and word in c2v:
                c2Vec = c2v[word]
            if gloVe is None:
                ten = c2Vec
            elif c2v is None:
                ten = glVec
            else:
                torch.cat((glVec, c2Vec), 0)
        if ten is not None:
            embedding_matrix[index] = ten
    return Embedding.from_pretrained(embedding_matrix, padding_idx=0, freeze=False)

In [111]:
import numpy as np
import csv
import chars2vec
def generateC2VRepresentation(vocab):
    li = np.array(list(vocab))
    print(li)
    f = open('data/c2v_100d.csv', 'w', encoding='utf-8')
    writer = csv.writer(f)
    writer.writerow(['word', 'c2v'])
    c2v_model = chars2vec.load_model('eng_100')
    vec = c2v_model.vectorize_words(li)
    for t, v in zip(li, vec):
        writer.writerow([t, v])
    f.close()

generateC2VRepresentation(train_dataset.textVocab.stoi.keys())

['<PAD>' '<UNK>' '@username' ... 'salsa' 'http://t.co/zkt4vi0wq3'
 'unethical']


In [159]:
import pandas as pd
def load_C2V_100():
    dev = "cuda:0"
    device = torch.device(dev)
    csv_data = pd.read_csv('data/c2v_100d.csv')
    c2v = {}
    for index, row in csv_data.iterrows():
        row[1] = row[1].replace('[', '')
        row[1] = row[1].replace(']', '')
        k = row[1].strip().split()
        c2v[row[0]] = torch.tensor([float(num) for num in k]).to(device)
    return c2v

c2v_100 = load_C2V_100()

In [160]:
gloVe_twitter = load_glove_twitter()


In [180]:
dev = "cuda:0"
device = torch.device(dev)
learning_rate=0.0001
batch_size=64
epoch = 20
bidirectional=True
num_layer=10
dropout=0.3
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                                  shuffle=True, collate_fn=pad_collate_fn)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=64,
                                  shuffle=True, collate_fn=pad_collate_fn)
test_dataloader = DataLoader(dataset=valid_dataset, batch_size=64,
                                  shuffle=True, collate_fn=pad_collate_fn)
em = create_embedding_t(train_dataset.textVocab.stoi, gloVe, None, False, 300)
b = LSTMModel(em, lstm=[300, 150], fc=[300, 150, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm= b.train_model(train_dataloader, valid_dataloader, num_iter=epoch, lr=learning_rate, early_stop=True)


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 0, eval loss= 0.612549901008606
Accuracy= 0.6876444143058067
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------
Epoch: 1, eval loss= 0.5355526804924011
Accuracy= 0.7699323136427567
f1= 0.5068314425701881
Precision= 0.7523512352626278
Recall= 0.39610883026472754
-----------------------------
Epoch: 2, eval loss= 0.5109564065933228
Accuracy= 0.7847473377536669
f1= 0.525785941218801
Precision= 0.8160195930486321
Recall= 0.40080348006378547
-----------------------------
Epoch: 3, eval loss= 0.4387589991092682
Accuracy= 0.8146348201727949
f1= 0.6655577712988366
Precision= 0.7616405534466127
Recall= 0.6036148690917761
-----------------------------


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 4, eval loss= 0.5126491189002991
Accuracy= 0.792674427365883
f1= 0.5470539238001103
Precision= 0.8134648661919693
Recall= 0.42197787481431953
-----------------------------
Epoch: 5, eval loss= 0.5148969888687134
Accuracy= 0.8009908077154912
f1= 0.5815882282733703
Precision= 0.8282287307363723
Recall= 0.45892192673367505
-----------------------------
Epoch: 6, eval loss= 0.44826287031173706
Accuracy= 0.8132628842676312
f1= 0.7141357797409176
Precision= 0.6804884258759921
Recall= 0.7610623801569992
-----------------------------
Epoch: 7, eval loss= 0.48051372170448303
Accuracy= 0.8156300231062888
f1= 0.666291888832704
Precision= 0.7594305359025935
Recall= 0.6040547129580077
-----------------------------
Epoch: 8, eval loss= 0.5105018019676208
Accuracy= 0.8154290988547318
f1= 0.6499794626223032
Precision= 0.7867452060421452
Recall= 0.5655538389085353
-----------------------------
Epoch: 9, eval loss= 0.4526931345462799
Accuracy= 0.8221694796061885
f1= 0.7069838753519928
Precision= 

In [182]:
bm.test_model(test_dataloader)

NameError: name 'data_loader' is not defined

In [184]:
dev = "cuda:0"
device = torch.device(dev)
learning_rate=0.0001
batch_size=64
epoch = 20
bidirectional=True
num_layer=10
dropout=0.3
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                                  shuffle=True, collate_fn=pad_collate_fn)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=64,
                                  shuffle=True, collate_fn=pad_collate_fn)
test_dataloader = DataLoader(dataset=valid_dataset, batch_size=64,
                                  shuffle=True, collate_fn=pad_collate_fn)
em = create_embedding_t(train_dataset.textVocab.stoi, None, c2v, False, 300)
b = LSTMModel(em, lstm=[300, 150], fc=[300, 150, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm= b.train_model(train_dataloader, valid_dataloader, num_iter=epoch, lr=learning_rate, early_stop=True)

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 0, eval loss= 0.6208815574645996
Accuracy= 0.6878390596745028
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 1, eval loss= 0.6211227178573608
Accuracy= 0.6878170835844887
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------


KeyboardInterrupt: 

In [ ]:
bm.test_model(test_dataloader)

In [186]:
em = create_embedding_t(train_dataset.textVocab.stoi, gloVe, c2v, False, 300)
b = LSTMModel(em, lstm=[600, 300], fc=[600, 300, 150, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm= b.train_model(train_dataloader, valid_dataloader, num_iter=30, lr=learning_rate, early_stop=False)

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 0, eval loss= 0.6207646131515503
Accuracy= 0.6876318565400844
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1, eval loss= 0.600813627243042
Accuracy= 0.7097837552742615
f1= 0.3057315197309169
Precision= 0.6027751784080899
Recall= 0.22031444008449538
-----------------------------
Epoch: 2, eval loss= 0.533913791179657
Accuracy= 0.7650002511553144
f1= 0.5572684537555835
Precision= 0.6749019870585885
Recall= 0.490222089305441
-----------------------------
Epoch: 3, eval loss= 0.5625306963920593
Accuracy= 0.7325604028531243
f1= 0.5913755668365943
Precision= 0.5665022961757451
Recall= 0.6385184503014517
-----------------------------
Epoch: 4, eval loss= 0.6211585998535156
Accuracy= 0.7155414908579466
f1= 0.5981699884387003
Precision= 0.5395653381038172
Recall= 0.6936927340598423
-----------------------------
Epoch: 5, eval loss= 0.6767557859420776
Accuracy= 0.6197853877838054
f1= 0.5864813246496002
Precision= 0.4484079898963564
Recall= 0.8688391989376155
-----------------------------
Epoch: 6, eval loss= 0.6235082745552063
Accuracy= 0.6575811231665661
f1= 0.5995195160917792
Precision= 0.47

In [193]:
em = create_embedding_t(train_dataset.textVocab.stoi, None, None, False, 300)
b = LSTMModel(em, lstm=[300, 150], fc=[300, 150, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm1= b.train_model(train_dataloader, valid_dataloader, num_iter=30, lr=learning_rate, early_stop=True)

TU sam


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 0, eval loss= 0.6213012933731079
Accuracy= 0.6878390596745028
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 1, eval loss= 0.5822506546974182
Accuracy= 0.7218768836648584
f1= 0.2987841584768808
Precision= 0.6645642683617368
Recall= 0.20431648715238587
-----------------------------
Epoch: 2, eval loss= 0.500987708568573
Accuracy= 0.7732977948563391
f1= 0.626275678879664
Precision= 0.639035052357684
Recall= 0.6308826367460254
-----------------------------
Epoch: 3, eval loss= 0.508029580116272
Accuracy= 0.743426009644364
f1= 0.636463108686294
Precision= 0.5679661549644363
Recall= 0.736024414679254
-----------------------------
Epoch: 4, eval loss= 0.4500810205936432
Accuracy= 0.8124466294956801
f1= 0.665955698238951
Precision= 0.7386575660692164
Recall= 0.6179684565053246
-----------------------------
Epoch: 5, eval loss= 0.4417513608932495
Accuracy= 0.817620428973277
f1= 0.6697359464367425
Precision= 0.7693543884975087
Recall= 0.6060112488425796
-----------------------------
Epoch: 6, eval loss= 0.4499738812446594
Accuracy= 0.7948500602772754
f1= 0.681131207097477
Precision= 0.655779583

In [194]:
em = create_embedding_t(train_dataset.textVocab.stoi, gloVe_twitter, c2v_100, True, 100)
b = LSTMModel(em, lstm=[300, 150], fc=[300, 150, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm2= b.train_model(train_dataloader, valid_dataloader, num_iter=30, lr=learning_rate, early_stop=False)

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 0, eval loss= 0.6211929321289062
Accuracy= 0.687829641350211
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------
Epoch: 1, eval loss= 0.7175054550170898
Accuracy= 0.5489721468756279
f1= 0.5304737669697197
Precision= 0.39620720998156245
Recall= 0.8318608848296547
-----------------------------
Epoch: 2, eval loss= 1.0302482843399048
Accuracy= 0.5206732218203738
f1= 0.5347379084215816
Precision= 0.38604117554930056
Recall= 0.8903030700025266
-----------------------------
Epoch: 3, eval loss= 1.1415334939956665
Accuracy= 0.4718078159533856
f1= 0.5234063116695067
Precision= 0.3654798136931292
Recall= 0.9393649559118908
-----------------------------
Epoch: 4, eval loss= 1.1481750011444092
Accuracy= 0.5084294002411092
f1= 0.5395342601072349
Precision= 0.3826219436681573
Recall= 0.9336750732817334
-----------------------------
Epoch: 5, eval loss= 1.2249408960342407
Accuracy= 0.472777903355435
f1= 0.5289592465928761
Precision= 0.36848996998739797
Recall= 0.9587362626011539


In [195]:
em = create_embedding_t(train_dataset.textVocab.stoi, gloVe_twitter, c2v, True, 300)
b = LSTMModel(em, lstm=[500, 250], fc=[500, 300, 150, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm4= b.train_model(train_dataloader, valid_dataloader, num_iter=30, lr=learning_rate, early_stop=False)

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 0, eval loss= 0.6508440971374512
Accuracy= 0.6878453385573639
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------
Epoch: 1, eval loss= 0.5459939241409302
Accuracy= 0.7596475035161744
f1= 0.4299772052006886
Precision= 0.7996096379007767
Recall= 0.3044469894214501
-----------------------------
Epoch: 2, eval loss= 0.5374710559844971
Accuracy= 0.7638072634116938
f1= 0.410619144793455
Precision= 0.889943741209564
Recall= 0.2742797546756246
-----------------------------
Epoch: 3, eval loss= 0.5907542109489441
Accuracy= 0.7814163903958208
f1= 0.49518104535435015
Precision= 0.85460209621602
Recall= 0.35750873593356314
-----------------------------
Epoch: 4, eval loss= 0.5528758764266968
Accuracy= 0.7685478199718706
f1= 0.4406302915209702
Precision= 0.8849885346720788
Recall= 0.30225435814408114
-----------------------------
Epoch: 5, eval loss= 0.7445747256278992
Accuracy= 0.7465842877235283
f1= 0.33461309185597454
Precision= 0.9054098854731765
Recall= 0.2091661432468114
-

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch: 27, eval loss= 1.151736855506897
Accuracy= 0.7256065400843882
f1= 0.24045820159366066
Precision= 0.8121559172192081
Recall= 0.14654336112965963
-----------------------------


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 28, eval loss= 1.2992726564407349
Accuracy= 0.7165366937914406
f1= 0.18099322926573905
Precision= 0.8067510548523206
Recall= 0.10536476939017847
-----------------------------


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 29, eval loss= 0.9639298915863037
Accuracy= 0.7301712879244525
f1= 0.26054055692805483
Precision= 0.8662145871006631
Recall= 0.15814609176407965
-----------------------------
Best accuracy = 0.7814163903958208
Best f1 = 0.49518104535435015


In [196]:
em = create_embedding_t(train_dataset.textVocab.stoi, gloVe, c2v_100, False, 100)
b = LSTMModel(em, lstm=[400, 200], fc=[400, 200, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm3= b.train_model(train_dataloader, valid_dataloader, num_iter=30, lr=learning_rate, early_stop=False)


Epoch: 0, eval loss= 0.5059820413589478
Accuracy= 0.7824053144464537
f1= 0.6130490127507543
Precision= 0.6900975888654051
Recall= 0.5658949717490103
-----------------------------
Epoch: 1, eval loss= 0.4838649034500122
Accuracy= 0.7966269841269842
f1= 0.5838536776363086
Precision= 0.7975987607468581
Recall= 0.47103198648713057
-----------------------------
Epoch: 2, eval loss= 0.4919525384902954
Accuracy= 0.7992044655414908
f1= 0.5902555405044508
Precision= 0.8002962392947499
Recall= 0.47644109522626993
-----------------------------
Epoch: 3, eval loss= 0.6499590277671814
Accuracy= 0.7529259594133012
f1= 0.3747518705192258
Precision= 0.8671730871097956
Recall= 0.247035601488136
-----------------------------
Epoch: 4, eval loss= 0.7010974287986755
Accuracy= 0.7542916164356037
f1= 0.3892716785709175
Precision= 0.8419064058304564
Recall= 0.2606832213945058
-----------------------------
Epoch: 5, eval loss= 0.6177404522895813
Accuracy= 0.7837709714687563
f1= 0.5134481640084462
Precision= 0

In [197]:
em = create_embedding_t(train_dataset.textVocab.stoi, gloVe, c2v_100, False, 100)
b = LSTMModel(em, lstm=[400, 200], fc=[400, 200, 75, 1], bidirectional=True, num_layer=num_layer, dropout=dropout).to(device)
bm3= b.train_model(train_dataloader, valid_dataloader, num_iter=30, lr=learning_rate, early_stop=False)

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: Un

Epoch: 0, eval loss= 0.6210296154022217
Accuracy= 0.687829641350211
f1= 0.0
Precision= 0.0
Recall= 0.0
-----------------------------


C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kruljac Jakov\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1, eval loss= 0.5793241262435913
Accuracy= 0.7214656168374523
f1= 0.21509726696364187
Precision= 0.8059774964838254
Recall= 0.12805359443242376
-----------------------------
Epoch: 2, eval loss= 0.5609234571456909
Accuracy= 0.7717186558167571
f1= 0.45498951264314835
Precision= 0.8755154128571848
Recall= 0.3166383244492125
-----------------------------
Epoch: 3, eval loss= 0.5128369331359863
Accuracy= 0.7942347297568817
f1= 0.5442660487170284
Precision= 0.8564434273015805
Recall= 0.4066514011733564
-----------------------------
Epoch: 4, eval loss= 0.4798426330089569
Accuracy= 0.8164117440225034
f1= 0.6420781749360388
Precision= 0.8067366949823743
Recall= 0.5438514376449957
-----------------------------
Epoch: 5, eval loss= 0.48269233107566833
Accuracy= 0.8118721117138838
f1= 0.6834577954287074
Precision= 0.7141524326931841
Recall= 0.6636719303897368
-----------------------------
Epoch: 6, eval loss= 0.4949195683002472
Accuracy= 0.8146348201727949
f1= 0.6776753945914681
Precision